# Hybrid Particle Swarm Optimization-Local Search

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [18]:
df = pd.read_csv("./data sets/wine/wine.data",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [19]:
df[0].value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

In [20]:
with open("./data sets/wine/wine.names") as f:
    print(f.read())

1. Title of Database: Wine recognition data
	Updated Sept 21, 1998 by C.Blake : Added attribute information

2. Sources:
   (a) Forina, M. et al, PARVUS - An Extendible Package for Data
       Exploration, Classification and Correlation. Institute of Pharmaceutical
       and Food Analysis and Technologies, Via Brigata Salerno, 
       16147 Genoa, Italy.

   (b) Stefan Aeberhard, email: stefan@coral.cs.jcu.edu.au
   (c) July 1991
3. Past Usage:

   (1)
   S. Aeberhard, D. Coomans and O. de Vel,
   Comparison of Classifiers in High Dimensional Settings,
   Tech. Rep. no. 92-02, (1992), Dept. of Computer Science and Dept. of
   Mathematics and Statistics, James Cook University of North Queensland.
   (Also submitted to Technometrics).

   The data was used with many others for comparing various 
   classifiers. The classes are separable, though only RDA 
   has achieved 100% correct classification.
   (RDA : 100%, QDA 99.4%, LDA 98.9%, 1NN 96.1% (z-transformed data))
   (All results usi

In [21]:
x = df.drop([0], axis=1)
x.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [22]:
y = df[[0]]
y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [26]:
class HPSO_LS():
    def __init__(self, x,y, NC=50, NP=5, c1=2, c2=2, v_max=4, v_min=-4,a=0.65, seed=101):
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        self.feature = x.columns
        self.NC = NC
        self.NP = NP
        self.c1 = c1
        self.c2 = c2
        self.v_max = v_max
        self.v_min = v_min
        self.a = a
        self.k = None
        self.similar = []
        self.dissimilar = [] 
        self.seed = seed
    
    def determining_k(self):
        random.seed(self.seed)
        f = len(self.feature)
        l_sf = []
        for sf in range(1,f):
            l= f - sf
            s =0
            for i in range (l):
#                 i +=1
                s += f - i
            l_sf.append(round(l / s,2))
        M = 0
        while(M<=2):
            e = (random.randint(15,70))/100
            M = int(e*f)
        k = random.choices(list(range(3,M+1)), weights= l_sf[3:M+1])
        self.k = k[0]
        return k[0]
    
    
    def grouping_features(self,plot = False):
        if plot: 
            corr = self.x.corr()
            plt.figure(figsize=(10,10))
            sns.heatmap(corr,annot=True)
            
            
        corr = []
        f = len(self.feature)
        for i in range(1,f):
            corr_i= x.corrwith(x[i])
            corr_i[i] = 0
            corr.append(((round(corr_i.apply(abs).sum()/(f-1),3)),i))
            corr.sort()
        self.dissimilar = corr[:len(corr)//2]
        self.similar = corr[len(corr)//2:]
        print("similar:   ",self.similar,"\ndissimilar :  ",self.dissimilar)
    
    def  initializing_particles(self):
        random.seed(self.seed)
        f = len(self.feature)
        k = self.determining_k()
        print(k,"k")
        particles = []
        sample = [1]*k + [0]*(f-k)
        for i in range(self.NP): 
            random.shuffle(sample)
            change = sample.copy()
            particles.append(change)   
        velocity = [[round(random.random(),3) for x in range(f) ]for y in range(self.NP)]
        
        return particles , velocity
    
    
    def fitness(self,P):
        score = []
        for p in P:
            model = KNeighborsClassifier()
            params = {
                'n_neighbors' : [1]
            }
            grid_model = GridSearchCV(model, params, cv=10)

            X_tr = pd.DataFrame(self.X_train)[[j for i,j in zip(p,pd.DataFrame(self.X_train).columns) if i==1]] 
            X_tst = pd.DataFrame(self.X_test)[[j for i,j in zip(p,pd.DataFrame(self.X_test).columns) if i==1]]
            grid_model.fit(X_tr,self.y_train)
            y_pred = grid_model.predict(X_tst)
            score.append(round(accuracy_score(self.y_test,y_pred),3))
            
        return score    
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    
    def update(self,P,V,p_best,g_best):
        f = len(self.feature)
        v_new = np.zeros_like(V)
        p_new = np.zeros_like(P)
        
        for i in range(self.NP):
            for d in range(f):
#                 print(P,"P")
                v_new[i][d] = V[i][d] + self.c1 *random.random()*(p_best[i][d] - P[i][d])* V[i][d] + self.c2*random.random()*(g_best[d]-P[i][d]) 
                if v_new[i][d] < self.v_min:
                    v_new[i][d] = self.v_min
                if v_new[i][d] > self.v_max:
                    v_new[i][d] = self.v_max
                
                rand = random.random()
                if self.sigmoid(v_new[i][d]) > rand:
                    p_new[i][d] = 1
                else:
                    p_new[i][d] = 0
                    
        return p_new , v_new            
                    
    def local_search(self,P):
        ns = int(self.a * self.k)
        nd = int((1-self.a) * self.k)
        p_new =[]
        for particle in range(self.NP):
            x_d =[]
            x_s =[]
            features = [j for j,i in enumerate(P[particle]) if i==1]
            for i,j in self.similar:
                if j in features:
                    x_s.append((i,j))
            for i,j in self.dissimilar:
                if j in features:
                    x_d.append((i,j)) 
                    
#             print(x_d,'B  xd')
#             print(x_s,'B  xs\n')                    
                    
                    
            l= ns-len(x_s)
            if l>0:
                for i in range(l):
                    for s in self.similar:
                        if s not in x_s:
                            x_s.append(s)
                            break
            elif l<0:
                for i in range(-l):
                    for x in x_s[::-1]:
                        x_s.pop()
                        break
            
            
            
            l= nd-len(x_d)            
            if l>0:
                for i in range(l):
                    for d in self.dissimilar:
                        if d not in x_d:
                            x_d.append(d)
                            break                                    
                

            elif l<0:
                for i in range(-l):
                    for x in x_d[::-1]:
                        x_d.pop()
                        break  
                        
#             print(x_d,'xd')
#             print(x_s,'xs\n')
            x_s = [i[1] for i in x_s]
            x_d = [i[1] for i in x_d]
            x = x_d + x_s
            sample = [0]* len(self.feature)
            for i in x:
                sample[i]=1
            particle = sample.copy()
            p_new.append(particle)
            
        return p_new
                
    def Run_HPSO_LS(self):
        self.determining_k()
        self.grouping_features()
        P,V= self.initializing_particles()
        scores = self.fitness(P)
        p_best = [(P[i],scores[i]) for i in range(len(P))]
        g_best = (P[scores.index(max(scores))],max(scores))
        print(g_best,"\n\n")          
#         return g_best ,p_best
        for i in range(self.NC):
#             print(g_best[0],"g best")
            p_new,v_new = self.update(P,V,[i[0] for i in p_best],g_best[0])
            p_new = self.local_search(p_new)
            scores_new = self.fitness(p_new)
            
            best_p = []
            for p in range(self.NP):
                if scores[p]>scores_new[p]:
                    best_p.append((P[p],scores[p]))
                else:
                    best_p.append((p_new[p],scores_new[p]))
                
                if scores_new[p] > g_best[1]:
                    g_best = [p_new[p],scores_new[p]]
           
            P = p_new
            V = v_new
            p_best = best_p.copy()
            for i in p_best:
                print(i)
            print(*g_best,"g_best \n")
                    
                    
            
            
                    

In [27]:
saman = HPSO_LS(x=x,y=y)

In [28]:
g, p = saman.Run_HPSO_LS()

similar:    [(0.3, 9), (0.319, 8), (0.325, 11), (0.384, 12), (0.409, 6), (0.447, 7)] 
dissimilar :   [(0.176, 3), (0.193, 5), (0.244, 10), (0.253, 1), (0.269, 2), (0.281, 4)]
6 k
([1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 0.981) 


([0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0], 0.963)
([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.944)
([0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1], 0.907)
([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.944)
([1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 0.981)
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.981 g_best 

([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1], 0.963)
([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.907)
([0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0], 0.907)
([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.944)
([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.981)
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.981 g_best 

([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1], 0.963)
([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.944)
([0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0], 0.907)
([0, 0, 0, 0, 0, 1, 0,

([0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0], 0.963)
([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], 0.907)
([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], 0.907)
([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0], 0.944)
([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0], 0.981)
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.981 g_best 

([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0], 0.963)
([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.926)
([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], 0.907)
([0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0], 0.944)
([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], 0.981)
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.981 g_best 

([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0], 0.963)
([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.926)
([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1], 0.944)
([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0], 0.944)
([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.981)
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0] 0.981 g_best 

([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.963)
([0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0.907)
([

TypeError: cannot unpack non-iterable NoneType object

In [1359]:
acc = []
# for i in range(100):
    
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

p = [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0]

model = KNeighborsClassifier()
params = {
    'n_neighbors' : [1]
}
grid_model = GridSearchCV(model, params, cv=10)

X_tr = pd.DataFrame(X_train)[[j for i,j in zip(p,pd.DataFrame(X_train).columns) if i==1]] 
X_tst = pd.DataFrame(X_test)[[j for i,j in zip(p,pd.DataFrame(X_test).columns) if i==1]]
grid_model.fit(X_tr,y_train)
y_pred = grid_model.predict(X_tst)
acc.append(accuracy_score(y_test,y_pred))
    
print(np.array(acc).mean())
print(np.array(acc).std())

1.0
0.0


In [1303]:
max(acc)

0.8769230769230769

In [1249]:
df = pd.read_csv("./data sets/glass/glass.data",header=None, index_col=0)
df.head()

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [1090]:
p[:-1]

[([0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0], 0.907),
 ([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1], 0.926),
 ([0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1], 0.889),
 ([0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1], 0.87),
 ([1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 0.907),
 ([1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], 0.981),
 ([0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0], 0.833),
 ([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0], 0.963),
 ([0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0], 0.778),
 ([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0], 0.907),
 ([0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0], 0.833),
 ([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1], 0.963),
 ([1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0], 0.815),
 ([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1], 0.926),
 ([0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0], 0.833),
 ([0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1], 0.907),
 ([1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0], 0.944),
 ([1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0], 0.87),
 ([0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], 0.87)]

In [1075]:
saman.determining_k()
saman.grouping_features()

similar:    [(0.3, 9), (0.319, 8), (0.325, 11), (0.384, 12), (0.409, 6), (0.447, 7)] 
dissimilar :   [(0.176, 3), (0.193, 5), (0.244, 10), (0.253, 1), (0.269, 2), (0.281, 4)]


In [1076]:
p , v = saman.initializing_particles()

6 k


In [1063]:
p

[[0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0],
 [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1],
 [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]]

In [1064]:
saman.local_search(p)

[(0.193, 5), (0.281, 4)] B  xd
[(0.3, 9), (0.325, 11), (0.409, 6), (0.447, 7)] B  xs

[(0.193, 5), (0.281, 4)] xd
[(0.3, 9), (0.325, 11), (0.409, 6)] xs

[(0.176, 3), (0.269, 2), (0.281, 4)] B  xd
[(0.3, 9), (0.325, 11), (0.384, 12)] B  xs

[(0.176, 3), (0.269, 2)] xd
[(0.3, 9), (0.325, 11), (0.384, 12)] xs

[(0.176, 3), (0.244, 10), (0.253, 1), (0.281, 4)] B  xd
[(0.319, 8), (0.384, 12)] B  xs

[(0.176, 3), (0.244, 10)] xd
[(0.319, 8), (0.384, 12), (0.3, 9)] xs

[(0.193, 5), (0.244, 10), (0.253, 1), (0.281, 4)] B  xd
[(0.319, 8), (0.384, 12)] B  xs

[(0.193, 5), (0.244, 10)] xd
[(0.319, 8), (0.384, 12), (0.3, 9)] xs

[(0.176, 3), (0.253, 1), (0.269, 2)] B  xd
[(0.325, 11), (0.409, 6)] B  xs

[(0.176, 3), (0.253, 1)] xd
[(0.325, 11), (0.409, 6), (0.3, 9)] xs

[(0.269, 2)] B  xd
[(0.3, 9), (0.384, 12), (0.409, 6), (0.447, 7)] B  xs

[(0.269, 2), (0.176, 3)] xd
[(0.3, 9), (0.384, 12), (0.409, 6)] xs

[(0.176, 3), (0.244, 10)] B  xd
[(0.3, 9), (0.319, 8), (0.325, 11), (0.409, 6)] B  xs

[

[[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]]

In [1050]:
score

[0.907,
 0.926,
 0.889,
 0.87,
 0.907,
 0.981,
 0.833,
 0.963,
 0.778,
 0.907,
 0.833,
 0.963,
 0.815,
 0.926,
 0.833,
 0.907,
 0.944,
 0.87,
 0.87,
 0.944]

In [994]:
saman.update(p,v,[])

NameError: name 'p_best' is not defined

In [1070]:
s = [1,2,3,5,8,2,3]
s.index(max(s))

4

In [1065]:
p = [0]*10
p[0] = 454
p

[454, 0, 0, 0, 0, 0, 0, 0, 0, 0]